# 로더 테스트
- 04_복지정책_v1.0.md 기준  
TextLoader: 0.0285초  
UnstructuredMarkdownLoader: 18.1923초  
성능 차이: 637.5배

In [ ]:
import time
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader

def performance_test(file_path: str):
    # TextLoader 테스트
    start = time.time()
    text_loader = TextLoader(file_path, encoding="UTF-8")
    text_docs = text_loader.load()
    text_time = time.time() - start
    print(f"TextLoader: {text_time:.4f}초")
    
    # UnstructuredMarkdownLoader 테스트
    start = time.time()
    unstructured_loader = UnstructuredMarkdownLoader(
        file_path, 
        mode="elements",
        strategy="fast"
    )
    unstructured_docs = unstructured_loader.load()
    unstructured_time = time.time() - start
    print(f"UnstructuredMarkdownLoader: {unstructured_time:.4f}초")
    
    print(f"성능 차이: {unstructured_time/text_time:.1f}배")

file_path = "./04_복지정책_v1.0.md"
performance_test(file_path)

# TextLoader + 헤더 기반 분할

In [3]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from typing import List
from langchain.schema import Document

def load_hr_document(file_path: str) -> List[Document]:
    """
    HR 정책 문서를 TextLoader + MarkdownHeaderTextSplitter로 로드하는 함수
    
    Args:
        file_path (str): 마크다운 파일 경로
    
    Returns:
        List[Document]: 분할된 문서 청크들
    """
    
    # 1. TextLoader로 문서 로드
    loader = TextLoader(file_path, encoding="utf-8")
    documents = loader.load()
    
    # 2. Document 객체에서 텍스트 내용 추출
    document_text = documents[0].page_content
    
    # 3. HR 문서 구조에 맞는 헤더 정의
    headers_to_split_on = [
        ("#", "문서제목"),          # # 가이다 플레이 스튜디오(GPS) 직원 복지제도 종합 안내서
        ("##", "정책대분류"),       # ## 1. 휴가 및 휴직 제도
        ("###", "정책세부항목"),    # ### 1.1 연차휴가
        # ("####", "세부절차"),       # #### **신청 절차**
    ]
    
    # 4. MarkdownHeaderTextSplitter로 구조적 분할
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on,
        strip_headers=False  # 헤더 정보 유지 (컨텍스트에 중요)
    )
    
    # 5. 문자열을 split_text에 전달 (Document가 아닌 str)
    md_header_splits = markdown_splitter.split_text(document_text)
    
    # 6. 긴 섹션을 위한 추가 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,        # HR Q&A에 적합한 크기
        chunk_overlap=200,      # 충분한 컨텍스트 오버랩
        separators=["\n\n", "\n", ". ", "? ", "! ", " ", ""]
    )
    
    # 7. 최종 분할 적용
    final_splits = text_splitter.split_documents(md_header_splits)
    
    return final_splits

In [4]:
# 복지정책 적용
file_path = "./04_복지정책_v1.0.md"
splits = load_hr_document(file_path)

len_splits = len(splits)
print(f"✅ 로딩 성공! 총 {len_splits}개 청크로 분할됨")
print("="*100)
for i in range(len_splits):
    print(f"메타데이터: {splits[i].metadata}")
    print(f"내용: {splits[i].page_content[:200]}...")
    print("\n")

✅ 로딩 성공! 총 15개 청크로 분할됨
메타데이터: {'문서제목': '가이다 플레이 스튜디오(GPS) 직원 복지제도 종합 안내서'}
내용: # 가이다 플레이 스튜디오(GPS) 직원 복지제도 종합 안내서
버전: v1.0
작성일: 2023-09-18
파일명: 04_복지정책_v1.0.md
본 문서는 가이다 플레이 스튜디오(GPS) 임직원을 위한 복지제도 종합 가이드입니다.
---...


메타데이터: {'문서제목': '가이다 플레이 스튜디오(GPS) 직원 복지제도 종합 안내서', '정책대분류': '1. 휴가 및 휴직 제도', '정책세부항목': '1.1 연차휴가'}
내용: ## 1. 휴가 및 휴직 제도  
### 1.1 연차휴가
- 연차휴가는 기본 15일입니다.
- 기존 근속자를 대상으로 매년 1월 1일 부여됩니다. (단, 전년도 출근율이 80% 이상이어야 발생)
- 발생한 연차는 해당년도 12월 31일까지 사용해야 하며, 미사용 연차는 이월되지 않습니다.
#### **신청 절차**
1. 전자결재(팀장) 승인 후 휴가 확정
...


메타데이터: {'문서제목': '가이다 플레이 스튜디오(GPS) 직원 복지제도 종합 안내서', '정책대분류': '1. 휴가 및 휴직 제도', '정책세부항목': '1.2 병가'}
내용: ### 1.2 병가
- 연간 5일 유급 병가를 사용할 수 있습니다. (5일 초과 시, 무급)
- 병가의 연간 누적일이 2일 초과 시, 의사 진단서 또는 소견서를 제출해야 합니다.
#### **신청 절차**
1. 전자결재(팀장) 승인 후 휴가 확정
2. 전자결재 승인 시 자동으로 Google Calendar와 연동되어 팀원들과 공유
3. 전자결제 시스템에서 ...


메타데이터: {'문서제목': '가이다 플레이 스튜디오(GPS) 직원 복지제도 종합 안내서', '정책대분류': '1. 휴가 및 휴직 제도', '정책세부항목': '1.3 가족돌봄휴가'}
내용: ### 1.3 가족돌봄휴가
- 법정 기준에 따라 연 10일 무급으로 사용할 수 있습니다.
- 자녀, 부모(

# 임베딩 모델 비교
- OpenAI text-embedding-3-small
- BAAI/bge-m3
- intfloat/multilingual-e5-large-instruct

In [ ]:
# 패키지 설치
# %pip install langchain langchain-openai langchain-community
# %pip install openai python-dotenv
# %pip install faiss-cpu

# 큰 모델 설치 시 메모리 부족 방지
# %pip install sentence-transformers transformers
# %pip install --no-cache-dir sentence-transformers
# %pip install --no-cache-dir transformers

In [ ]:
# 설치 검증 스크립트
try:
    import langchain
    import langchain_openai
    import langchain_community
    import openai
    import sentence_transformers
    import transformers
    import faiss
    import torch
    from dotenv import load_dotenv
    
    print("✅ 모든 패키지 설치 완료!")
    print(f"PyTorch 버전: {torch.__version__}")
    print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
    
except ImportError as e:
    print(f"❌ 패키지 설치 실패: {e}")

In [ ]:
import os
import time
import gc
from typing import List, Dict, Any
from dotenv import load_dotenv

# LangChain imports
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

# .env 파일에서 API 키 로드
load_dotenv()

def get_hr_test_queries() -> List[str]:
    """
    HR 문서 관련 테스트 쿼리 반환
    """
    return [
        "연차휴가는 몇 일인가요?",
        "복지포인트는 어떻게 사용하나요?",
        "교육비 지원 신청 방법을 알려주세요",
        "육아 지원금은 얼마나 받을 수 있나요?",
        "건강검진은 언제부터 받을 수 있나요?",
        "장기 근속자 가산일 규정",
        "병가 사용 시 필요한 서류",
        "동아리 활동비 지원 한도"
    ]

def test_search_performance(vector_store: FAISS, model_name: str, test_queries: List[str]) -> Dict:
    """
    단일 모델의 검색 성능 테스트
    """
    print(f"\n🔍 {model_name} 검색 성능 테스트")
    print("=" * 50)
    
    model_results = []
    total_time = 0
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n질문 {i}: {query}")
        
        start_time = time.time()
        # 유사도 검색 (상위 3개 결과)
        search_results = vector_store.similarity_search(query, k=3)
        search_time = time.time() - start_time
        total_time += search_time
        
        print(f"⏱️  검색 시간: {search_time:.3f}초")
        print("📋 검색 결과:")
        
        query_result = {
            "query": query,
            "search_time": search_time,
            "results": []
        }
        
        for j, doc in enumerate(search_results, 1):
            # 결과 미리보기 (처음 100자)
            preview = doc.page_content[:100].replace('\n', ' ')
            print(f"  {j}. {preview}...")
            
            query_result["results"].append({
                "rank": j,
                "content": doc.page_content,
                "metadata": doc.metadata
            })
        
        model_results.append(query_result)
    
    result = {
        "model_name": model_name,
        "total_time": total_time,
        "avg_time": total_time / len(test_queries),
        "queries": model_results
    }
    
    print(f"\n📊 {model_name} 전체 성능:")
    print(f"  총 검색 시간: {total_time:.3f}초")
    print(f"  평균 검색 시간: {total_time/len(test_queries):.3f}초")
    
    return result

def test_openai_embedding(documents: List[Document], test_queries: List[str]) -> Dict:
    """
    OpenAI text-embedding-3-small 테스트
    """
    model_name = "OpenAI text-embedding-3-small"
    print(f"\n🚀 {model_name} 테스트 시작")
    
    try:
        # 1. 임베딩 모델 로드
        print("📥 임베딩 모델 로드 중...")
        embeddings = OpenAIEmbeddings(
            model="text-embedding-3-small",
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )
        print("✅ OpenAI 임베딩 로드 완료")
        
        # 2. 벡터스토어 생성
        print("🔄 벡터스토어 생성 중...")
        start_time = time.time()
        vector_store = FAISS.from_documents(
            documents=documents,
            embedding=embeddings
        )
        creation_time = time.time() - start_time
        print(f"✅ 벡터스토어 생성 완료 ({creation_time:.2f}초)")
        
        # 3. 검색 성능 테스트
        result = test_search_performance(vector_store, model_name, test_queries)
        result["creation_time"] = creation_time
        
        # 4. 메모리 정리
        del vector_store, embeddings
        gc.collect()
        print(f"🧹 {model_name} 메모리 정리 완료")
        
        return result
        
    except Exception as e:
        print(f"❌ {model_name} 테스트 실패: {e}")
        return {"model_name": model_name, "error": str(e)}

def test_bge_m3_embedding(documents: List[Document], test_queries: List[str]) -> Dict:
    """
    BAAI/bge-m3 테스트
    """
    model_name = "BAAI/bge-m3"
    print(f"\n🚀 {model_name} 테스트 시작")
    
    try:
        # 1. 임베딩 모델 로드
        print("📥 임베딩 모델 로드 중...")
        embeddings = HuggingFaceEmbeddings(
            model_name="BAAI/bge-m3",
            model_kwargs={'device': 'cpu'},  # GPU 사용시 'cuda'로 변경
            encode_kwargs={'normalize_embeddings': True}
        )
        print("✅ BGE-M3 임베딩 로드 완료")
        
        # 2. 벡터스토어 생성
        print("🔄 벡터스토어 생성 중...")
        start_time = time.time()
        vector_store = FAISS.from_documents(
            documents=documents,
            embedding=embeddings
        )
        creation_time = time.time() - start_time
        print(f"✅ 벡터스토어 생성 완료 ({creation_time:.2f}초)")
        
        # 3. 검색 성능 테스트
        result = test_search_performance(vector_store, model_name, test_queries)
        result["creation_time"] = creation_time
        
        # 4. 메모리 정리
        del vector_store, embeddings
        gc.collect()
        print(f"🧹 {model_name} 메모리 정리 완료")
        
        return result
        
    except Exception as e:
        print(f"❌ {model_name} 테스트 실패: {e}")
        return {"model_name": model_name, "error": str(e)}

def test_e5_large_embedding(documents: List[Document], test_queries: List[str]) -> Dict:
    """
    intfloat/multilingual-e5-large-instruct 테스트
    """
    model_name = "multilingual-e5-large-instruct"
    print(f"\n🚀 {model_name} 테스트 시작")
    
    try:
        # 1. 임베딩 모델 로드
        print("📥 임베딩 모델 로드 중...")
        embeddings = HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-large-instruct",
            model_kwargs={'device': 'cpu'},  # GPU 사용시 'cuda'로 변경
            encode_kwargs={'normalize_embeddings': True}
        )
        print("✅ E5-Large 임베딩 로드 완료")
        
        # 2. 벡터스토어 생성
        print("🔄 벡터스토어 생성 중...")
        start_time = time.time()
        vector_store = FAISS.from_documents(
            documents=documents,
            embedding=embeddings
        )
        creation_time = time.time() - start_time
        print(f"✅ 벡터스토어 생성 완료 ({creation_time:.2f}초)")
        
        # 3. 검색 성능 테스트
        result = test_search_performance(vector_store, model_name, test_queries)
        result["creation_time"] = creation_time
        
        # 4. 메모리 정리
        del vector_store, embeddings
        gc.collect()
        print(f"🧹 {model_name} 메모리 정리 완료")
        
        return result
        
    except Exception as e:
        print(f"❌ {model_name} 테스트 실패: {e}")
        return {"model_name": model_name, "error": str(e)}

def compare_all_models(documents: List[Document]) -> List[Dict]:
    """
    모든 모델을 순차적으로 테스트하고 결과 비교
    """
    print("🚀 임베딩 모델 순차 비교 테스트 시작")
    print("="*60)
    
    test_queries = get_hr_test_queries()
    results = []
    
    # 각 모델을 순차적으로 테스트 (메모리 절약)
    print(f"\n📋 {len(test_queries)}개 테스트 쿼리 준비 완료")
    
    # 1. OpenAI 테스트
    openai_result = test_openai_embedding(documents, test_queries)
    results.append(openai_result)
    
    # 2. BGE-M3 테스트  
    bge_result = test_bge_m3_embedding(documents, test_queries)
    results.append(bge_result)
    
    # 3. E5-Large 테스트
    e5_result = test_e5_large_embedding(documents, test_queries)
    results.append(e5_result)
    
    # 4. 결과 비교
    print_comparison_summary(results)
    
    return results

def test_single_model(documents: List[Document], model_choice: str) -> Dict:
    """
    단일 모델만 테스트 (메모리 제약이 심할 때)
    
    Args:
        documents: HR 문서 청크들
        model_choice: 'openai', 'bge', 'e5' 중 선택
    """
    test_queries = get_hr_test_queries()
    
    if model_choice.lower() == 'openai':
        return test_openai_embedding(documents, test_queries)
    elif model_choice.lower() == 'bge':
        return test_bge_m3_embedding(documents, test_queries)
    elif model_choice.lower() == 'e5':
        return test_e5_large_embedding(documents, test_queries)
    else:
        raise ValueError("model_choice는 'openai', 'bge', 'e5' 중 하나여야 합니다.")

def print_comparison_summary(results: List[Dict]):
    """
    모델별 성능 비교 요약 출력
    """
    print("\n" + "="*60)
    print("🏆 임베딩 모델 성능 비교 요약")
    print("="*60)
    
    # 성공한 모델들만 필터링
    valid_results = [r for r in results if "error" not in r]
    
    if not valid_results:
        print("❌ 성공한 테스트가 없습니다.")
        return
    
    # 성능 데이터 준비
    performance_data = []
    for result in valid_results:
        performance_data.append({
            "모델": result["model_name"],
            "벡터스토어 생성": f"{result.get('creation_time', 0):.2f}초",
            "평균 검색 시간": f"{result['avg_time']:.3f}초",
            "총 검색 시간": f"{result['total_time']:.3f}초"
        })
    
    # 검색 속도순 정렬
    performance_data.sort(key=lambda x: float(x["평균 검색 시간"].replace("초", "")))
    
    print("\n📊 성능 순위 (빠른 순):")
    for i, data in enumerate(performance_data, 1):
        print(f"  {i}. {data['모델']}")
        print(f"     벡터스토어 생성: {data['벡터스토어 생성']}")
        print(f"     평균 검색 시간: {data['평균 검색 시간']}")
        print()
    
    print("💡 MVP 개발 권장사항:")
    if performance_data:
        fastest_model = performance_data[0]["모델"]
        print(f"  • 검색 속도 1위: {fastest_model}")
        print(f"  • 10일 MVP 개발용 추천: {fastest_model}")
        print(f"  • Pinecone 연동 시 이 모델 사용 권장")
    
    # 에러 발생한 모델들 표시
    error_results = [r for r in results if "error" in r]
    if error_results:
        print("\n⚠️  테스트 실패한 모델:")
        for result in error_results:
            print(f"  • {result['model_name']}: {result['error']}")

# 사용 예시 함수들
def quick_test_all(documents: List[Document]):
    """
    빠른 전체 모델 테스트 (추천)
    """
    return compare_all_models(documents)

def quick_test_one(documents: List[Document], model: str):
    """
    단일 모델 빠른 테스트
    """
    return test_single_model(documents, model)

if __name__ == "__main__":
    print("🔧 임베딩 모델 비교 테스트 도구")
    print("="*50)
    print("사용법:")
    print("1. 전체 모델 테스트: quick_test_all(documents)")
    print("2. 단일 모델 테스트: quick_test_one(documents, 'openai')")
    print("3. 개별 함수 호출:")
    print("   - test_openai_embedding(documents, queries)")
    print("   - test_bge_m3_embedding(documents, queries)")  
    print("   - test_e5_large_embedding(documents, queries)")
    
    # 실제 사용 예시:
    # documents = load_hr_policy_document("04_복지정책_v1.0.md")
    # results = quick_test_all(documents)
    # 또는
    # result = quick_test_one(documents, 'openai')